## Домашняя работа
ФИО:Владимиров Никита Алексеевич
Группа: РИ-150950

## Задание 1
Реализуйте метакласс ThreadSafeSingleton, который обеспечивает создание только одного экземпляра класса, даже в многопоточной среде.
Используйте `from threading import Lock`


In [6]:
from threading import Lock

class DatabasePool(metaclass=type):
    pass

class ThreadSafeSingleton(type):
    _instances = {}
    _locks = {}

    def __call__(cls, *args, **kwargs):
        if cls in cls._instances:
            return cls._instances[cls]

        lock = cls._locks.get(cls)
        if lock is None:
            lock = Lock()
            cls._locks[cls] = lock

        with lock:
            if cls not in cls._instances:
                instance = super().__call__(*args, **kwargs)
                cls._instances[cls] = instance
        return cls._instances[cls]

class DatabasePool(metaclass=ThreadSafeSingleton):
    def __init__(self):
        self._counter = 0
        self._connections = []

    def get_connection(self):
        conn = object()
        self._connections.append(conn)
        self._counter += 1
        return conn

### проверка задания 1

In [7]:

# Создайте 3 экземпляра DatabasePool
pool1 = DatabasePool()
pool2 = DatabasePool()
pool3 = DatabasePool()

# Убедитесь, что это один и тот же объект
assert pool1 is pool2 is pool3

# Проверьте, что соединения разделяются между экземплярами
conn1 = pool1.get_connection()
conn2 = pool2.get_connection()
assert conn1 != conn2


## Задание 2

Создайте метакласс, который считает, сколько раз создавался каждый класс.

Требования:
1. Метакласс должен иметь атрибут _counters
1. При создании экземпляра класса счетчик должен увеличиваться
1. Добавьте метод get_count(), который возвращает количество созданных экземпляров

In [8]:
class InstanceCounter(type):
    _counters = {}

    def __call__(cls, *args, **kwargs):
        cls._counters[cls] = cls._counters.get(cls, 0) + 1
        return super().__call__(*args, **kwargs)

    def get_count(cls):
        return cls._counters.get(cls, 0)

class User(metaclass=InstanceCounter):
    def __init__(self, name):
        self.name = name

class Product(metaclass=InstanceCounter):
    def __init__(self, name):
        self.name = name

### проверка задания 2

In [9]:
# Проверка
user1 = User("Alice")
user2 = User("Bob")
product1 = Product("Laptop")

print(User.get_count())    # Должно быть 2
print(Product.get_count()) # Должно быть 1

2
1


## Задание 3

Создайте метакласс, который автоматически добавляет метод describe() в каждый класс.

Требования:
1. Метод describe() должен возвращать строку с именем класса
1. Используйте метакласс для создания классов Car и Book

In [10]:
class AutoDescribe(type):
    def __new__(cls, name, bases, attrs):
        if 'describe' not in attrs:
            def describe(self):
                return f"Это объект класса {self.__class__.__name__}"
            attrs['describe'] = describe
        return super().__new__(cls, name, bases, attrs)

class Car(metaclass=AutoDescribe):
    def __init__(self, brand):
        self.brand = brand

class Book(metaclass=AutoDescribe):
    def __init__(self, title):
        self.title = title

### проверка задания 3

In [11]:
car = Car("Toyota")
book = Book("Python для начинающих")

print(car.describe())  # Должно быть "Это объект класса Car"
print(book.describe()) # Должно быть "Это объект класса Book"

Это объект класса Car
Это объект класса Book


## Задание 4

Создайте метакласс, который проверяет, что у класса есть метод save(). Можно использовать `__new__`

Требования:
1. Если у класса нет метода save(), метакласс должен выдать ошибку
1. Создайте класс User с методом save()
1. Попробуйте создать класс Message без метода save() (должна быть ошибка)

In [18]:
class SaveMeta(type):
    def __new__(cls, name, bases, attrs):
        return super().__new__(cls, name, bases, attrs)

class User(metaclass=SaveMeta):
    def __init__(self, name):
        self.name = name

    def save(self):
        print(f"User {self.name} saved")

class Message(metaclass=SaveMeta):
    def __init__(self, text):
        self.text = text

### проверка задания 4

In [20]:
# Проверка
user = User("Alice")
user.save()  # Должно работать

# Этот код должен вызвать ошибку:
# class Message(metaclass=SaveMeta):
#     def __init__(self, text):
#         self.text = text

User Alice saved
